<a href="https://colab.research.google.com/github/bavilaa/nlp_got/blob/master/scraping_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#                 , _ ,
#                ( o o )
#               /'` ' `'\
#               |'''''''|
#               |\\'''//|
#                  """
#BAA

In [0]:
#Activating Google Drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
#Scraping scripts of Game of Thrones

#importing mechanize, useful library to download plain html from  webpages.

import mechanize
from lxml import etree
import re

#beautiful soup, another library to parse html
from bs4 import BeautifulSoup

#pandas

import pandas as pd





In [0]:
#Loading the webpage

browser = mechanize.Browser()
url = "https://www.springfieldspringfield.co.uk/episode_scripts.php?tv-show=game-of-thrones"
browser = mechanize.Browser()
html = browser.open(url).get_data()


In [0]:
#Getting all the links from all the seasons of Game of trhones
#using lxml



#links are in the form of //*[@id="content_container"]/div[2]/div[2]/div[season]/a[episode]
#starting season in = 3

#loading html into xlml
tree = etree.HTML(html)

#list for the links
links = []

for s in range (3,14):
  for e in range(1,30):

    route = '//*[@id="content_container"]/div[2]/div[2]/div[' + str(s) + ']/a[' + str(e) + ']'

    try:
      route_link = tree.xpath(route)
      link = route_link[0].attrib['href']
      #print (link)
      #saving the link into the list:
      links.append(link)

    except:
      continue



In [0]:
#Simple function to clean the text

def clean_html(html):
    """
    Removes most common formatting of a string that represents a price
    leaving it only with its numbers.
    """

    blacklist = ['\r', '\n', '\t']

    for item in blacklist:
        html = html.replace(item, '')

    return html


In [0]:
#Now , with all the links is necessary to load the link and extract the script from the webpage

scripts = []

j = 0
url_script = links[j]

for j in links:
  
  url_script = 'https://www.springfieldspringfield.co.uk/' + j
  #print (url_script)

  #Episode
  episode = url_script[-6:]

  #loading the html with the script
  try:
    html_script = browser.open(url_script).get_data()
    status = 1
    soup = BeautifulSoup(html_script , 'html.parser')

  #extracting the script
    scripts_route = soup.findAll("div", {"class": "scrolling-script-container"})
    script = clean_html(scripts_route[0].text)
    status = 1
    scripts.append ([episode,script,status])

  except:
    script = ""
    status = 0
    scripts.append ([episode,script,status])

  #parsing html with BS
  soup = BeautifulSoup(html_script , 'html.parser')

  #extracting the script
  #print (script[0:100])
  





In [0]:
#Renaiming the columns
df = pd.DataFrame(scripts)
df.rename(columns = {0:'Season_Episode', 1:'Script', 
                              2:'Status' }, inplace = True)

In [0]:
#Creating Season and Episde Column, useles to merge with rating database
df['Season'] = df.Season_Episode.str[2:3]
df['Episode'] = df.Season_Episode.str[4:6]
df[["Season", "Episode"]] = df[["Season", "Episode"]].apply(pd.to_numeric)

In [0]:
#Saving dataframe into a csv file

route = '/content/drive/My Drive/Colab Notebooks/Project_NLP/'


df.to_csv(route + "scripts_bd.csv")

In [12]:
df.head(5)

,Season_Episode,Script,Status,Season,Episode
0,s01e01,"Easy, boy. What do you exp...",1,1,1
1,s01e02,"You need to drink, child. ...",1,1,2
2,s01e03,"Welcome, Lord Stark. Grand...",1,1,3
3,s01e04,The little lord's been dre...,1,1,4
4,s01e05,Does Ser Hugh have any fam...,1,1,5
